# 在 Azure Kubernetes Service (AKS) 上部署 Phi-4 模型完整教程

本教程将指导您使用 Azure CLI 命令行工具在 AKS 上部署 Microsoft Phi-4 模型，实现一个可扩展的模型推理服务。

## 目录
1. [前置条件](#前置条件)
2. [环境准备](#环境准备)
3. [创建 AKS 集群](#创建-aks-集群)
4. [部署 Azure 机器学习扩展](#部署-azure-机器学习扩展)
5. [创建 Azure 机器学习工作区](#创建-azure-机器学习工作区)
6. [将 AKS 集群附加到工作区](#将-aks-集群附加到工作区)
7. [部署 Phi-4 模型](#部署-phi-4-模型)
8. [测试模型端点](#测试模型端点)
9. [监控和日志](#监控和日志)
10. [清理资源](#清理资源)

## 前置条件

### 系统要求
- Azure 订阅（如果没有，请创建[免费账户](https://azure.microsoft.com/free/)）
- Azure CLI 2.50.0 或更高版本
- kubectl 命令行工具
- Python 3.8+ 环境
- 至少 16GB RAM 的开发机器

### 安装必要工具

首先，我们需要安装和配置必要的工具。以下命令将安装 Azure CLI、kubectl 和相关扩展：

In [ ]:
# 安装 Azure CLI (如果尚未安装)
!curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

# 验证安装
!az --version

# 安装 kubectl
!az aks install-cli

# 安装 k8s-extension 扩展
!az extension add --name k8s-extension

# 更新扩展到最新版本
!az extension update --name k8s-extension

### 登录 Azure

使用以下命令登录到 Azure 并设置默认订阅：

In [ ]:
# 登录 Azure
!az login

# 设置默认订阅（如果有多个订阅）
# !az account set --subscription "<subscription-id>"

# 验证当前订阅
!az account show

## 环境准备

### 定义环境变量

我们需要设置一些环境变量来管理我们的资源：

In [ ]:
import os
import random

# 设置环境变量
os.environ["RESOURCE_GROUP"] = "phi4-aks-rg"
os.environ["LOCATION"] = "eastus"
os.environ["AKS_CLUSTER_NAME"] = "phi4-aks-cluster"
os.environ["ML_WORKSPACE_NAME"] = "phi4-ml-workspace"
os.environ["EXTENSION_NAME"] = "phi4-ml-extension"
os.environ["COMPUTE_NAME"] = "phi4-k8s-compute"
os.environ["ENDPOINT_NAME"] = "phi4-endpoint"
os.environ["DEPLOYMENT_NAME"] = "phi4-deployment"
os.environ["ACR_NAME"] = f"phi4acr{random.randint(0, 999999)}"
os.environ["STORAGE_ACCOUNT"] = f"phi4storage{random.randint(0, 999999)}"

# 显示配置
print(f"Resource Group: {os.environ['RESOURCE_GROUP']}")
print(f"Location: {os.environ['LOCATION']}")
print(f"AKS Cluster: {os.environ['AKS_CLUSTER_NAME']}")
print(f"ML Workspace: {os.environ['ML_WORKSPACE_NAME']}")

## 创建 AKS 集群

### 1. 创建资源组

首先，我们需要创建一个资源组来管理所有相关资源：

In [ ]:
!az group create \
    --name $RESOURCE_GROUP \
    --location $LOCATION

### 2. 创建 AKS 集群

创建一个适合运行 Phi-4 模型的 AKS 集群：

In [ ]:
!az aks create \
    --resource-group $RESOURCE_GROUP \
    --name $AKS_CLUSTER_NAME \
    --node-count 2 \
    --node-vm-size Standard_NC6s_v3 \
    --enable-managed-identity \
    --generate-ssh-keys \
    --enable-addons monitoring \
    --enable-cluster-autoscaler \
    --min-count 1 \
    --max-count 3 \
    --node-osdisk-size 100

### 3. 获取 AKS 凭据

配置 kubectl 以连接到新创建的集群：

In [ ]:
# 配置 kubectl 连接到集群
!az aks get-credentials \
    --resource-group $RESOURCE_GROUP \
    --name $AKS_CLUSTER_NAME

# 验证连接
!kubectl get nodes
!kubectl get pods --all-namespaces

### 4. 安装 NVIDIA 设备插件

如果使用 GPU 节点，需要安装 NVIDIA 设备插件：

In [ ]:
# 应用 NVIDIA 设备插件 DaemonSet
!kubectl apply -f https://raw.githubusercontent.com/NVIDIA/k8s-device-plugin/v0.14.1/nvidia-device-plugin.yml

# 验证 GPU 是否可用
!kubectl get nodes -o json | jq '.items[].status.allocatable."nvidia.com/gpu"'

## 部署 Azure 机器学习扩展

### 1. 注册必要的资源提供程序

首先需要注册 KubernetesConfiguration 提供程序：

In [ ]:
# 注册 KubernetesConfiguration 提供程序
!az provider register --namespace Microsoft.KubernetesConfiguration

# 等待注册完成
!az provider show -n Microsoft.KubernetesConfiguration -o table

### 2. 部署 ML 扩展

部署 Azure Machine Learning 扩展到 AKS 集群：

In [ ]:
!az k8s-extension create \
    --name $EXTENSION_NAME \
    --extension-type Microsoft.AzureML.Kubernetes \
    --config enableTraining=True \
    --config enableInference=True \
    --config inferenceRouterServiceType=LoadBalancer \
    --config allowInsecureConnections=False \
    --config InferenceRouterHA=True \
    --config installNvidiaDevicePlugin=True \
    --config installDcgmExporter=True \
    --cluster-type managedClusters \
    --cluster-name $AKS_CLUSTER_NAME \
    --resource-group $RESOURCE_GROUP \
    --scope cluster

### 3. 验证扩展部署

检查扩展的部署状态和相关资源：

In [ ]:
# 检查扩展状态
!az k8s-extension show \
    --name $EXTENSION_NAME \
    --cluster-type managedClusters \
    --cluster-name $AKS_CLUSTER_NAME \
    --resource-group $RESOURCE_GROUP

# 检查 azureml 命名空间中的 pods
!kubectl get pods -n azureml
!kubectl get services -n azureml

## 创建 Azure 机器学习工作区

### 1. 创建存储账户和容器注册表

首先创建必要的依赖资源：

In [ ]:
# 创建存储账户
!az storage account create \
    --name $STORAGE_ACCOUNT \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --sku Standard_LRS \
    --kind StorageV2

# 创建容器注册表
!az acr create \
    --name $ACR_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --sku Basic \
    --admin-enabled true

### 2. 创建 ML 工作区

创建 Azure Machine Learning 工作区：

In [ ]:
# 创建 ML 工作区
!az ml workspace create \
    --name $ML_WORKSPACE_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --storage-account $STORAGE_ACCOUNT \
    --container-registry $ACR_NAME

## 将 AKS 集群附加到工作区

### 1. 获取 AKS 资源 ID 并创建命名空间

In [ ]:
# 获取 AKS 资源 ID
%%bash
AKS_RESOURCE_ID=$(az aks show \
    --name $AKS_CLUSTER_NAME \
    --resource-group $RESOURCE_GROUP \
    --query id -o tsv)

echo "AKS Resource ID: $AKS_RESOURCE_ID"

# 创建 Kubernetes 命名空间
!kubectl create namespace phi4-inference

### 2. 附加集群到工作区并配置权限

In [ ]:
%%bash
# 附加集群到工作区
!az ml compute attach \
    --resource-group $RESOURCE_GROUP \
    --workspace-name $ML_WORKSPACE_NAME \
    --type Kubernetes \
    --name $COMPUTE_NAME \
    --resource-id $AKS_RESOURCE_ID \
    --identity-type SystemAssigned \
    --namespace phi4-inference

# 获取计算标识
COMPUTE_IDENTITY=$(az ml compute show \
    --name $COMPUTE_NAME \
    --workspace-name $ML_WORKSPACE_NAME \
    --resource-group $RESOURCE_GROUP \
    --query identity.principal_id -o tsv)

# 获取 ACR ID
ACR_ID=$(az acr show \
    --name $ACR_NAME \
    --resource-group $RESOURCE_GROUP \
    --query id -o tsv)

# 授予 ACR 拉取权限
!az role assignment create \
    --assignee $COMPUTE_IDENTITY \
    --role "AcrPull" \
    --scope $ACR_ID

# 获取存储账户 ID
STORAGE_ID=$(az storage account show \
    --name $STORAGE_ACCOUNT \
    --resource-group $RESOURCE_GROUP \
    --query id -o tsv)

# 授予存储账户访问权限
!az role assignment create \
    --assignee $COMPUTE_IDENTITY \
    --role "Storage Blob Data Contributor" \
    --scope $STORAGE_ID

## 部署 Phi-4 模型

### 1. 创建部署配置文件

首先创建必要的目录结构和配置文件：

创建目录结构


In [5]:
!mkdir "phi4-deployment/model" "phi4-deployment/script"

创建评分脚本

In [6]:
%%writefile phi4-deployment/script/score.py
import json
import logging
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

def init():
    """
    初始化模型和分词器
    """
    global model, tokenizer
    
    # 设置日志
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)
    
    # 模型路径
    model_path = os.getenv('AZUREML_MODEL_DIR', '/var/azureml-app/azureml-models/phi4/1')
    
    logger.info(f"Loading model from: {model_path}")
    
    # 加载模型和分词器
    model_name = "microsoft/phi-4"
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
    
    # 设置 pad token
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    logger.info("Model loaded successfully")

def run(raw_data):
    """
    处理推理请求
    """
    try:
        # 解析输入数据
        data = json.loads(raw_data)
        prompt = data.get("prompt", "")
        max_tokens = data.get("max_tokens", 200)
        temperature = data.get("temperature", 0.7)
        top_p = data.get("top_p", 0.95)
        
        # 准备输入
        inputs = tokenizer(prompt, return_tensors="pt", padding=True)
        
        # 生成响应
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                top_p=top_p,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id
            )
        
        # 解码输出
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 移除原始提示词
        response = response[len(prompt):].strip()
        
        return json.dumps({
            "response": response,
            "model": "phi-4",
            "usage": {
                "prompt_tokens": len(inputs["input_ids"][0]),
                "completion_tokens": len(outputs[0]) - len(inputs["input_ids"][0]),
                "total_tokens": len(outputs[0])
            }
        })
        
    except Exception as e:
        error_message = f"Error during inference: {str(e)}"
        logging.error(error_message)
        return json.dumps({"error": error_message})

Writing phi4-deployment/script/score.py


创建环境配置文件


In [4]:
%%writefile phi4-deployment/model/conda.yml
name: phi4-inference
channels:
  - pytorch
  - nvidia
  - conda-forge
  - defaults
dependencies:
  - python=3.10
  - pip
  - pytorch=2.1.0
  - pytorch-cuda=12.1
  - pip:
    - transformers>=4.36.0
    - accelerate>=0.25.0
    - sentencepiece
    - protobuf
    - einops
    - flash-attn
    - azureml-defaults
    - azureml-inference-server-http

Writing conda.yml


创建部署配置文件

In [ ]:
%%writefile phi4-deployment/deployment.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineDeployment.schema.json
name: phi4-deployment
endpoint_name: phi4-endpoint
model:
  name: phi4
  version: 1
  path: ./model
code_configuration:
  code: ./script/
  scoring_script: score.py
environment:
  name: phi4-env
  version: 1
  conda_file: ./model/conda.yml
  image: mcr.microsoft.com/azureml/curated/acpt-pytorch-2.1-cuda12.1:latest
instance_type: Standard_NC6s_v3
instance_count: 1
request_settings:
  request_timeout_ms: 90000
  max_concurrent_requests_per_instance: 1
  max_queue_wait_ms: 30000
liveness_probe:
  initial_delay: 600
  period: 100
  timeout: 70
  success_threshold: 1
  failure_threshold: 3
readiness_probe:
  initial_delay: 600
  period: 100
  timeout: 70
  success_threshold: 1
  failure_threshold: 3
environment_variables:
  TRANSFORMERS_CACHE: /tmp/.cache/huggingface
  HF_HOME: /tmp/.cache/huggingface
  CUDA_VISIBLE_DEVICES: 0

### 2. 创建在线端点并部署模型

创建端点配置文件


In [ ]:
%%writefile endpoint.yml
$schema: https://azuremlschemas.azureedge.net/latest/managedOnlineEndpoint.schema.json
name: $ENDPOINT_NAME
compute: $COMPUTE_NAME
auth_mode: key
traffic:
  phi4-deployment: 100

# 创建端点
!az ml online-endpoint create \
    --name $ENDPOINT_NAME \
    --resource-group $RESOURCE_GROUP \
    --workspace-name $ML_WORKSPACE_NAME \
    --file endpoint.yml

# 部署模型
!az ml online-deployment create \
    --name $DEPLOYMENT_NAME \
    --endpoint-name $ENDPOINT_NAME \
    --resource-group $RESOURCE_GROUP \
    --workspace-name $ML_WORKSPACE_NAME \
    --file phi4-deployment/deployment.yml \
    --all-traffic

## 测试模型端点

### 1. 获取端点信息

In [ ]:

%%bash
# 获取评分 URI
SCORING_URI=$(az ml online-endpoint show \
    --name $ENDPOINT_NAME \
    --resource-group $RESOURCE_GROUP \
    --workspace-name $ML_WORKSPACE_NAME \
    --query scoring_uri -o tsv)

# 获取主密钥
API_KEY=$(az ml online-endpoint get-credentials \
    --name $ENDPOINT_NAME \
    --resource-group $RESOURCE_GROUP \
    --workspace-name $ML_WORKSPACE_NAME \
    --query primaryKey -o tsv)

print(f"Scoring URI: {SCORING_URI}")
print(f"API Key: {API_KEY}")

### 2. 测试端点

使用 Python 发送测试请求：

In [ ]:
import requests
import json

# 准备请求头
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}

# 测试数据
test_prompts = [
    {
        "prompt": "Explain quantum computing in simple terms:",
        "max_tokens": 150,
        "temperature": 0.7
    },
    {
        "prompt": "Write a Python function to calculate fibonacci numbers:",
        "max_tokens": 200,
        "temperature": 0.5
    }
]

# 发送请求
for i, data in enumerate(test_prompts):
    print(f"\n--- Test {i+1} ---")
    print(f"Prompt: {data['prompt']}")
    
    try:
        response = requests.post(SCORING_URI, headers=headers, json=data)
        response.raise_for_status()
        
        result = response.json()
        print(f"Response: {result.get('response', 'No response')}")
        print(f"Tokens used: {result.get('usage', {}).get('total_tokens', 'N/A')}")
        
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response content: {e.response.text}")

## 清理资源

完成测试后，清理所有创建的资源以避免产生不必要的费用：

In [ ]:
# 删除在线端点
!az ml online-endpoint delete \
    --name $ENDPOINT_NAME \
    --resource-group $RESOURCE_GROUP \
    --workspace-name $ML_WORKSPACE_NAME \
    --yes

# 分离计算
!az ml compute detach \
    --name $COMPUTE_NAME \
    --resource-group $RESOURCE_GROUP \
    --workspace-name $ML_WORKSPACE_NAME \
    --yes

# 删除 ML 扩展
!az k8s-extension delete \
    --name $EXTENSION_NAME \
    --cluster-type managedClusters \
    --cluster-name $AKS_CLUSTER_NAME \
    --resource-group $RESOURCE_GROUP \
    --yes

# 删除 AKS 集群
!az aks delete \
    --name $AKS_CLUSTER_NAME \
    --resource-group $RESOURCE_GROUP \
    --yes

# 删除 ML 工作区
!az ml workspace delete \
    --name $ML_WORKSPACE_NAME \
    --resource-group $RESOURCE_GROUP \
    --yes

# 删除资源组（这将删除所有剩余资源）
!az group delete \
    --name $RESOURCE_GROUP \
    --yes